In [1]:
!pip install ray==2.41.0

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 25.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.0/430.0 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 33.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
!pip install numpy mlflow tensorflow "ray[serve,default,client]"

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 119.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 87.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.6/620.6 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 121.9 MB/s eta 0:00:0000:01
   ━━━━━━━

In [1]:
import mlflow
import mlflow.tensorflow
import numpy as np

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# -------------------
# Prepare Data
# -------------------
data = load_diabetes()
X = data.data
y = data.target.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -------------------
# Define Model
# -------------------
def create_model(input_dim):
    model = keras.Sequential([
        layers.Dense(64, activation="relu", input_shape=(input_dim,)),
        layers.Dense(32, activation="relu"),
        layers.Dense(1)  # regression output
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mse"])
    return model

input_dim = X_train.shape[1]
epochs = 50
batch_size = 32

mlflow.set_experiment("Diabetes_Prediction_TensorFlow")

with mlflow.start_run():
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("optimizer", "adam")
    mlflow.log_param("loss_fn", "mse")
    mlflow.log_param("input_features", input_dim)

    model = create_model(input_dim)

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )

    # Evaluation
    loss, mse = model.evaluate(X_test, y_test, verbose=0)
    rmse = np.sqrt(mse)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)

2025-08-27 12:00:23.577265: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-27 12:00:23.626853: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-27 12:00:25.157402: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025/08/27 12:00:27 INFO mlflow.tracking.fluent: Experiment with name 'Diabetes_Prediction_TensorFlow' does not exist. Creating a new experiment.
2025/08/27 12:00:27 WARNING mlflow.tracking.context.registry: Encountered unexpected error during resolving tags: 'getpwuid(): uid not found: 1001'
2025/08/27 12:00:27 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is pr

🏃 View run classy-wren-479 at: http://ai-starter-kit-mlflow:5000/#/experiments/1/runs/7ca28b8521a049dc8a014d4235909db3
🧪 View experiment at: http://ai-starter-kit-mlflow:5000/#/experiments/1


In [2]:
import numpy as np
import mlflow.tensorflow
import tensorflow as tf
from starlette.requests import Request
from typing import Dict

from ray import serve
import ray


@serve.deployment(
    ray_actor_options={
        "runtime_env": {
            "pip": ["tensorflow"]
        },
    }
)
class TensorFlowMLflowDeployment:
    def __init__(self):
        print("Loading model...")
        self.model = model
        print("Model loaded successfully.")

    async def __call__(self, input_data) -> Dict:
        try:
            if isinstance(input_data, Request):
                data = await input_data.json()
            else:
                data = input_data
            features = data.get("features", None)
            if features is None:
                return {"error": "Missing 'features' in request"}
            X = np.array(features).reshape(1, -1)

            # Make prediction with TensorFlow model
            prediction = self.model.predict(X).flatten().tolist()

            return {"prediction": prediction}
        except Exception as e:
            return {"error": str(e)}


# Bind and deploy
app = TensorFlowMLflowDeployment.bind()
handle = serve.run(app, route_prefix="/predict")

/opt/bitnami/miniconda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-27 12:00:35,162	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-08-27 12:00:35,747	INFO worker.py:1514 -- Using address ray://ai-starter-kit-kuberay-head-svc:10001 set in the environment variable RAY_ADDRESS
2025-08-27 12:00:35,748	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.
2025-08-27 12:00:40,043	WARNING utils.py:1591 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.41.0
    Python: 3.11.11
This process on Ray Client was started with:
    Ray: 2.41.0
    Python: 3.11.

(ServeReplica:default:TensorFlowMLflowDeployment pid=2265) Loading model...
(ServeReplica:default:TensorFlowMLflowDeployment pid=2265) Model loaded successfully.
(ServeReplica:default:TensorFlowMLflowDeployment pid=2265, ip=10.23.2.212) Loading model...
(ServeReplica:default:TensorFlowMLflowDeployment pid=2265, ip=10.23.2.212) Model loaded successfully.


INFO 2025-08-27 12:02:08,716 serve 124 -- Application 'default' is ready at http://127.0.0.1:8000/predict.
INFO 2025-08-27 12:02:08,718 serve 124 -- Deployed app 'default' successfully.
(ServeReplica:default:TensorFlowMLflowDeployment pid=2265) INFO 2025-08-27 05:02:08,967 default_TensorFlowMLflowDeployment 0zpbmyix 63d6b1b4-67f7-4a5c-ad92-6d7989e3cf5a -- CALL __call__ OK 150.7ms
(ServeReplica:default:TensorFlowMLflowDeployment pid=2265, ip=10.23.2.212) INFO 2025-08-27 05:02:08,967 default_TensorFlowMLflowDeployment 0zpbmyix 63d6b1b4-67f7-4a5c-ad92-6d7989e3cf5a -- CALL __call__ OK 150.7ms


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step3.2.212) 


(ProxyActor pid=1775) INFO 2025-08-27 05:02:09,833 proxy 10.23.2.213 -- Proxy starting on node 45179940bb5a43115519b525607191a8f1a059b70c5c61c14cee8a0f (HTTP port: 8000).
(ProxyActor pid=1775, ip=10.23.2.212) INFO 2025-08-27 05:02:09,833 proxy 10.23.2.213 -- Proxy starting on node 45179940bb5a43115519b525607191a8f1a059b70c5c61c14cee8a0f (HTTP port: 8000).
(ProxyActor pid=1775) INFO 2025-08-27 05:02:09,992 proxy 10.23.2.213 -- Got updated endpoints: {Deployment(name='TensorFlowMLflowDeployment', app='default'): EndpointInfo(route='/predict', app_is_cross_language=False)}.
(ProxyActor pid=1775, ip=10.23.2.212) INFO 2025-08-27 05:02:09,992 proxy 10.23.2.213 -- Got updated endpoints: {Deployment(name='TensorFlowMLflowDeployment', app='default'): EndpointInfo(route='/predict', app_is_cross_language=False)}.


In [3]:
json_data = {"features": [0.0380759, 0.0506801, 0.0616962, 0.0218724, -0.0442235, -0.0348208, -0.0434008, -0.00259226, 0.0199084, -0.0176461]}
response = handle.remote(json_data)
await response

{'prediction': [179.46218872070312]}